# Python pandas

In [2]:

import pandas as pd
import statsmodels.api as sm

## Импорт и создание DataFrame 

Выполните код ниже, чтобы загрузить из файла `anes96.csv` данные [Американских национальных исследований выборов 1996 года](https://www.statsmodels.org/dev/datasets/generated/anes96.html).

In [ ]:
df = pd.read_csv('anes96.csv')

: 

Информация о содержимом датасета:

```plain
    Variables name definitions::

            popul - Census place population in 1000s
            TVnews - Number of times per week that respondent watches TV news.
            PID - Party identification of respondent.
                0 - Strong Democrat
                1 - Weak Democrat
                2 - Independent-Democrat
                3 - Independent-Indpendent
                4 - Independent-Republican
                5 - Weak Republican
                6 - Strong Republican
            age : Age of respondent.
            educ - Education level of respondent
                1 - 1-8 grades
                2 - Some high school
                3 - High school graduate
                4 - Some college
                5 - College degree
                6 - Master's degree
                7 - PhD
            income - Income of household
                1  - None or less than $2,999
                2  - $3,000-$4,999
                3  - $5,000-$6,999
                4  - $7,000-$8,999
                5  - $9,000-$9,999
                6  - $10,000-$10,999
                7  - $11,000-$11,999
                8  - $12,000-$12,999
                9  - $13,000-$13,999
                10 - $14,000-$14.999
                11 - $15,000-$16,999
                12 - $17,000-$19,999
                13 - $20,000-$21,999
                14 - $22,000-$24,999
                15 - $25,000-$29,999
                16 - $30,000-$34,999
                17 - $35,000-$39,999
                18 - $40,000-$44,999
                19 - $45,000-$49,999
                20 - $50,000-$59,999
                21 - $60,000-$74,999
                22 - $75,000-89,999
                23 - $90,000-$104,999
                24 - $105,000 and over
            vote - Expected vote
                0 - Clinton
                1 - Dole
            The following 3 variables all take the values:
                1 - Extremely liberal
                2 - Liberal
                3 - Slightly liberal
                4 - Moderate
                5 - Slightly conservative
                6 - Conservative
                7 - Extremely Conservative
            selfLR - Respondent's self-reported political leanings from "Left"
                to "Right".
            ClinLR - Respondents impression of Bill Clinton's political
                leanings from "Left" to "Right".
            DoleLR  - Respondents impression of Bob Dole's political leanings
                from "Left" to "Right".
            logpopul - log(popul + .1)
```

## 1. Основные свойства DataFrame

DataFrame (`df`) содержит данные о зарегистрированных избирателях в США, включая демографические данные и политические взгляды. Используя `pandas`, выведите первые 5 строк DataFrame, чтобы понять как примерно выглядят данные. Далее ответьте на следующие вопросы:

* Сколько записей в DataFrame?
* Сколько параметров было оценено в исследовании (сколько столбцов)?
* Каков минимальный и максимальный возраст избирателя?
* Сколько раз в неделю в среднем респонденты смотрят новости (округлите до десятых)?
* Проверьте данные на пропущенные значения.

In [ ]:
# Вывод первых 5 строк DataFrame
print("Первые 5 строк DataFrame:")
print(df.head())

# Количество записей в DataFrame
print("Количество записей в DataFrame:", len(df))

# Количество параметров (столбцов) в DataFrame
print("Количество параметров (столбцов) в DataFrame:", len(df.columns))

# Минимальный и максимальный возраст избирателя
min_age = df['age'].min()
max_age = df['age'].max()
print("Минимальный возраст избирателя:", min_age)
print("Максимальный возраст избирателя:", max_age)

# Среднее количество просмотров новостей в неделю
mean_news_views = df['news'].mean()
print("Среднее количество просмотров новостей в неделю:", round(mean_news_views, 1))

# Проверка на пропущенные значения
missing_values = df.isnull().sum().sum()
if missing_values == 0:
    print("Пропущенных значений нет.")
else:
    print("Количество пропущенных значений:", missing_values)


: 

## 2. Обработка данных

Необходимо настроить датасет для дальнейшего использования. Сделайте следующее:

*   Переименуйте столбец `educ` в `education`.
*   Создайте новый столбец `party` на основе ответов каждого респондента в `PID`. `party` должен быть равен `Democrat`, если респондент выбрал или Strong Democrat, или Weak Democrat. `party` должен быть равен `Republican`, если респондент выбрал Strong или Weak Republican в `PID` и `Independent` во всех остальных случаях.
*   Создайте новый столбец `age_group`, который объединяет респондентов по следующим категориям на основе их возраста `age`: 18-24, 25-34, 35-44, 45-54, 55-64, и 65+.

In [ ]:
# Переименование столбца 'educ' в 'education'
df.rename(columns={'educ': 'education'}, inplace=True)

# Функция для создания столбца 'party' на основе столбца 'PID'
def assign_party(row):
    if row['PID'] in ['Strong Democrat', 'Weak Democrat']:
        return 'Democrat'
    elif row['PID'] in ['Strong Republican', 'Weak Republican'] or row['PID'] == 'Republican':
        return 'Republican'
    else:
        return 'Independent'

# Создание столбца 'party'
df['party'] = df.apply(assign_party, axis=1)

# Функция для создания столбца 'age_group' на основе столбца 'age'
def assign_age_group(age):
    if age >= 18 and age <= 24:
        return '18-24'
    elif age >= 25 and age <= 34:
        return '25-34'
    elif age >= 35 and age <= 44:
        return '35-44'
    elif age >= 45 and age <= 54:
        return '45-54'
    elif age >= 55 and age <= 64:
        return '55-64'
    else:
        return '65+'

# Создание столбца 'age_group'
df['age_group'] = df['age'].apply(assign_age_group)

# Вывод первых 5 строк DataFrame для проверки изменений
print(df.head())

: 

## 3. Фильтрация данных

* Используя фильтрацию найдите всех респондентов, кто считал Билла Клинтона умеренным или консерватором (`ClinLR` равно 4 или выше). Сколько респондентов в этом подмножестве?
* Среди всех респондентов, сколько имеют доход ниже $50,000 и хотя бы посещали колледж?

In [ ]:
# Найдем всех респондентов, которые считали Билла Клинтона умеренным или консерватором (ClinLR равно 4 или выше)
clinton_moderate_conservative = df[(df['ClinLR'] >= 4)]

# Подсчитаем количество респондентов в этом подмножестве
num_clinton_moderate_conservative = len(clinton_moderate_conservative)

print("Количество респондентов, считающих Билла Клинтона умеренным или консерватором:", num_clinton_moderate_conservative)

# Найдем всех респондентов, у которых доход ниже $50,000 и хотя бы посещали колледж (income < 50000 и education > 12)
low_income_college_attended = df[(df['income'] < 50000) & (df['education'] > 12)]

# Подсчитаем количество респондентов в этом подмножестве
num_low_income_college_attended = len(low_income_college_attended)

print("Количество респондентов с доходом ниже $50,000 и хотя бы посещавших колледж:", num_low_income_college_attended)

: 

## 4. Вычисления на основе данных

Для каждого из следующих сочетаний выберите группу, которая более вероятно проголосует за Билла Клинтона. Вы можете это вычислить используя долю каждой группы, которая намерена голосовать за Клинтона (`vote`). Какое сочетание имеет наименьшую разницу, а какое наибольшую?


*   Демократы или республиканцы
*   Люди младше 44 или люди старше 44 лет и старше
*   Люди, которые смотрят новости по меньшей мере 6 дней в неделю, или, которые смотрят менее 3 дней в неделю
*   Люди живущие там, где население меньше чем у среднего респондента, или там, где равно и больше


In [ ]:
# Your code here
clinton_votes = df.groupby('vote')['vote'].value_counts()
clinton_percentages = clinton_votes / df['vote'].value_counts()
pairs = [
    ('Democrat', 'Republican'),
    ('Age < 44', 'Age >= 44'),
    ('Watch News >= 6 days', 'Watch News < 3 days'),
    ('Population < Mean', 'Population >= Mean'),
]
min_difference = None
max_difference = None

for pair1, pair2 in pairs:
    group1_data = df[df['group'] == pair1]
    group2_data = df[df['group'] == pair2]
    group1_percentage = clinton_percentages['Clinton']
    group2_percentage = clinton_percentages['Clinton']
    difference = abs(group1_percentage - group2_percentage)
    if min_difference is None or difference < min_difference:
        min_difference = difference
        min_pair = (pair1, pair2)
    if max_difference is None or difference > max_difference:
        max_difference = difference
        max_pair = (pair1, pair2)
print(f"Пара с наименьшей разницей в доле голосов за Клинтона: {min_pair}, разница: {min_difference:.2f}")
print(f"Пара с наибольшей разницей в доле голосов за Клинтона: {max_pair}, разница: {max_difference:.2f}")

: 

## 5. Группировка данных

Используя метод `groupby()` объедините респондентов по `age_group`. Какая возрастная группа сама консервативная? Какая меньше всех смотрит новости?

Далее, рассчитайте 5-процентильные группы в зависимости от дохода. Сгруппируйте набор данных по этим процентилям. Какая группа доходов является наиболее либеральной? Какая самой консервативной? Старейшей? Наиболее образованной?

In [ ]:
# Your code here
age_groups = df.groupby('age_group')
most_conservative_age_group = age_groups['political_view'].idxmin()
print(f"Самая консервативная возрастная группа: {most_conservative_age_group}")

least_news_watching_age_group = age_groups['watch_news'].idxmin()
print(f"Группа, которая наименее смотрит новости: {least_news_watching_age_group}")

income_percentiles = pd.qcut(df['income'], 5)
income_groups = df.groupby(income_percentiles)
most_liberal_income_group = income_groups['political_view'].idxmax()
print(f"Самая либеральная группа дохода: {most_liberal_income_group}")

most_conservative_income_group = income_groups['political_view'].idxmin()
print(f"Самая консервативная группа дохода: {most_conservative_income_group}")

oldest_income_group = income_groups['age'].idxmax()
print(f"Самая старая группа дохода: {oldest_income_group}")

most_educated_income_group = income_groups['education'].idxmax()
print(f"Самая образованная группа дохода: {most_educated_income_group}")

## 6. Голосование вне логики

Вам интересно узнать больше о респондентах, чьи политические взгляды сильно отличаются от кандидата, за которого они планируют голосовать. Используя `selfLR`, `vote`, `ClinLR`, и `DoleLR`, ответьте на следующие вопросы:

*   Какая наибольшая известная разница между политическими взглядами респондента и его кандидата?
*   Сколько респондентов показали такую разницу?
*   Сделайте отдельный DataFrame `sway`, который включает только избирателей, которые демонстрируют разницу более $|3|$.
*   В `sway`, больше респондентов голосующих за более либерального или более консервативного кандидата, чем они сами?
*   В `sway`, какой кандидат наиболее популярен?

In [ ]:
# Your code here
df['ClinLR_diff'] = abs(df['selfLR'] - df['ClinLR'])
df['DoleLR_diff'] = abs(df['selfLR'] - df['DoleLR'])
max_diff = max(df['ClinLR_diff'].max(), df['DoleLR_diff'].max())
print(f"Наибольшая разница: {max_diff}")

diff_3_or_more = (df['ClinLR_diff'] >= 3) | (df['DoleLR_diff'] >= 3)
count_diff_3_or_more = diff_3_or_more.sum()
print(f"Количество респондентов с разницей >= 3: {count_diff_3_or_more}")

sway = df[diff_3_or_more]

clinton_votes_sway = sway[sway['vote'] == 'Clinton'].shape[0]
dole_votes_sway = sway[sway['vote'] == 'Dole'].shape[0]

more_votes = 'Clinton' if clinton_votes_sway > dole_votes_sway else 'Dole'
less_votes = 'Clinton' if clinton_votes_sway < dole_votes_sway else 'Dole'

print(f"В sway больше голосов за: {more_votes} ({clinton_votes_sway} голосов)")
print(f"В sway меньше голосов за: {less_votes} ({dole_votes_sway} голосов)")

most_popular_candidate = sway['vote'].mode()[0]
print(f"Самый популярный кандидат в sway: {most_popular_candidate}")